# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [176]:
! pip install cartopy
! pip install geoviews


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\evilp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\evilp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [177]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [178]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bonthe,7.5264,-12.5050,26.12,81,78,2.59,SL,1700953168
1,1,nemuro,43.3236,145.5750,-0.76,66,23,7.86,JP,1700953168
2,2,pont rouge,48.8833,-72.0825,-3.84,81,98,5.61,CA,1700953169
3,3,cangucu,-31.3950,-52.6756,17.56,88,85,2.76,BR,1700953169
4,4,sagua la grande,22.8067,-80.0756,24.12,84,20,4.07,CU,1700953169


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [179]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"  
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [180]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather_df = city_data_df.loc[(city_data_df["Humidity"] < 75) | (city_data_df["Max Temp"] < 30)]

# Drop any rows with null values
best_city_df = best_weather_df.dropna()

# Display sample data
best_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bonthe,7.5264,-12.5050,26.12,81,78,2.59,SL,1700953168
1,1,nemuro,43.3236,145.5750,-0.76,66,23,7.86,JP,1700953168
2,2,pont rouge,48.8833,-72.0825,-3.84,81,98,5.61,CA,1700953169
3,3,cangucu,-31.3950,-52.6756,17.56,88,85,2.76,BR,1700953169
4,4,sagua la grande,22.8067,-80.0756,24.12,84,20,4.07,CU,1700953169


### Step 3: Create a new DataFrame called `hotel_df`.

In [181]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ' '

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bonthe,SL,7.5264,-12.5050,81,
1,nemuro,JP,43.3236,145.5750,66,
2,pont rouge,CA,48.8833,-72.0825,81,
3,cangucu,BR,-31.3950,-52.6756,88,
4,sagua la grande,CU,22.8067,-80.0756,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [182]:
# Set parameters to search for a hotel
radius = 10000
limit = 20

# set up a parameters dictionary
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

#Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]

    latitude = row["Lat"]
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bonthe - nearest hotel: No hotel found
nemuro - nearest hotel: No hotel found
pont rouge - nearest hotel: No hotel found
cangucu - nearest hotel: No hotel found
sagua la grande - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
rongelap - nearest hotel: No hotel found
tralee - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
banjarmasin - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
cadiz - nearest hotel: No hotel found
hastings - nearest hotel: No hotel found
nabire - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
vilyuysk - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bonthe,SL,7.5264,-12.5050,81,No hotel found
1,nemuro,JP,43.3236,145.5750,66,No hotel found
2,pont rouge,CA,48.8833,-72.0825,81,No hotel found
3,cangucu,BR,-31.3950,-52.6756,88,No hotel found
4,sagua la grande,CU,22.8067,-80.0756,84,No hotel found
...,...,...,...,...,...,...
562,krasnyy yar,RU,46.5331,48.3456,80,No hotel found
563,maasim,PH,5.8575,125.0061,75,No hotel found
564,qingdao,CN,36.0986,120.3719,93,No hotel found
565,chalmette,US,29.9427,-89.9634,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [183]:
#Configure the map plot
country_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
country_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)